### 1. 필요한 라이브러리 불러오기

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
import json
from tqdm import tqdm
import re

### 2. 데이터 불러오기

In [2]:
df = pd.read_csv('C:\Kcoding\팀프로젝트 데이터\내가 담당한것\Place_API\place_location.csv', encoding='utf-8')
df

,Unnamed: 0,address,place_name,food,lat,lng
0,0,"서울특별시 종로구 경희궁길 36, 지하1층 204호 (신문로2가)",커먼맨션,경양식,197287.415745,452294.360874
1,1,"서울특별시 강북구 4.19로 55, 1,2층 (수유동)",귀한돌짜장 매운갈비찜,한식,200682.917084,460435.453596
2,2,"서울특별시 도봉구 도봉로 493-4, 1층 (쌍문동)",할배골,한식,203012.670169,460828.952571
3,3,"서울특별시 중랑구 용마산로118길 67, 1층 (망우동)",망우로스팅랩,기타,209147.166204,455438.575009
4,4,"서울특별시 중랑구 양원역로 82, 101호 (망우동)",본죽&비빔밥 서울양원역점,기타,209279.447244,456130.266637
...,...,...,...,...,...,...
12105,12105,"서울특별시 송파구 백제고분로 198, 1층 104호 (삼전동)",청년어부 삼전점,일식,207761.510501,444639.252711
12106,12106,"서울특별시 송파구 올림픽로35가길 10, 1층 118호 (신천동, 잠실 더샵스타파크)",진전복 삼계탕 잠실직영점,한식,208972.490842,446087.944327
12107,12107,"서울특별시 송파구 백제고분로40길 5, 2층 12호 (석촌동)",알이랑 고기랑 석촌점,한식,209324.864323,444648.957259
12108,12108,"서울특별시 강동구 동남로81길 58, 1층 103호 (고덕동)",남도에서,한식,213510.482226,450829.307236


### 3. 불러온 데이터 위도, 경도 값 수정

In [3]:
# 위도, 경도 컬럼 삭제
df.drop(['lat'], axis=1, inplace=True)
df.drop(['lng'], axis=1, inplace=True)

In [4]:
# 카카오 API 사용을 위한 주소 세부정보 삭제한 값을 addr_list 변수에 저장
addr_list = []
for i in df['address']:
    addr = i.split(',')[0]
    addr_list.append(addr)

In [5]:
# 기존 address 컬럼을 삭제 후 새로운 address 컬럼에 addr_list 변수 데이터 삽입
# (다른 데이터와 컬럼명을 통일 하기 위하여 부득이하게 이 작업을 진행하였습니다.)
df.drop(['address'], axis=1, inplace=True)
df['address'] = addr_list

In [7]:
# 결과물 확인
df

,Unnamed: 0,place_name,food,address
0,0,커먼맨션,경양식,서울특별시 종로구 경희궁길 36
1,1,귀한돌짜장 매운갈비찜,한식,서울특별시 강북구 4.19로 55
2,2,할배골,한식,서울특별시 도봉구 도봉로 493-4
3,3,망우로스팅랩,기타,서울특별시 중랑구 용마산로118길 67
4,4,본죽&비빔밥 서울양원역점,기타,서울특별시 중랑구 양원역로 82
...,...,...,...,...
12105,12105,청년어부 삼전점,일식,서울특별시 송파구 백제고분로 198
12106,12106,진전복 삼계탕 잠실직영점,한식,서울특별시 송파구 올림픽로35가길 10
12107,12107,알이랑 고기랑 석촌점,한식,서울특별시 송파구 백제고분로40길 5
12108,12108,남도에서,한식,서울특별시 강동구 동남로81길 58


In [8]:
# 카카오 API KEY 불러오기
with open('C:/Kcoding/KakaoAPI.txt', "r") as f:
    api_key = f.read()

In [11]:
# 위도, 경도 값을 받아온뒤 각 리스트에 저장
# 값을 찾을수 없을때, 해당 주소를 출력하고 None값이 입력되게끔 세팅
lat_list, lng_list = [], []

for i in tqdm(df['address']):
    addr = f'{(i)}'
    quote(addr)
    search_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{search_url}?query={quote(addr)}'
    result = requests.get(url, headers = {'Authorization': f'KakaoAK {api_key}'}).json()
    if not result['documents']:
        print(f"No result found for {addr}")
        lat_list.append(None)
        lng_list.append(None)
    else:
        lng = float(result['documents'][0]['x'])
        lat = float(result['documents'][0]['y'])
        lat_list.append(lat)
        lng_list.append(lng)

 59%|█████▉    | 7118/12110 [23:18<12:08,  6.85it/s]  

No result found for 서울특별시 영등포구 가마산로79길 46


100%|██████████| 12110/12110 [41:56<00:00,  4.81it/s] 


In [12]:
# 데이터프레임에 위도, 경도 값 추가
df['lat'] = lat_list
df['lng'] = lng_list

In [24]:
# None 값으로 입력된 데이터 확인
df[df['lat'].isnull()]

,Unnamed: 0,place_name,food,address,lat,lng
7116,7116,홍성한우,한식,서울특별시 영등포구 가마산로79길 46,NaN,NaN


In [25]:
# None 값 데이터 삭제
df.drop([7116], axis=0, inplace=True)

In [91]:
# 불필요하게 생성된 컬럼 삭제
df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [92]:
# 임시 데이터 저장
df.to_csv('place_location_temporary.csv', index=False, encoding='utf-8-sig')

### 4. 업종(food) 확인 및 변경
- 가게명을 바탕으로 업종을 바꿈.

In [180]:
# 임시 데이터 불러오기
df = pd.read_csv('C:\Kcoding\팀프로젝트 데이터\내가 담당한것\Place_API\place_location_api.csv')
df

,place_name,food,address,lat,lng
0,커먼맨션,경양식,서울특별시 종로구 경희궁길 36,37.572920,126.970063
1,귀한돌짜장 매운갈비찜,한식,서울특별시 강북구 4.19로 55,37.646420,127.008572
2,할배골,한식,서울특별시 도봉구 도봉로 493-4,37.649833,127.034923
3,망우로스팅랩,기타,서울특별시 중랑구 용마산로118길 67,37.601238,127.104370
4,본죽&비빔밥 서울양원역점,기타,서울특별시 중랑구 양원역로 82,37.607469,127.105874
...,...,...,...,...,...
12104,청년어부 삼전점,일식,서울특별시 송파구 백제고분로 198,37.503938,127.088562
12105,진전복 삼계탕 잠실직영점,한식,서울특별시 송파구 올림픽로35가길 10,37.516940,127.102633
12106,알이랑 고기랑 석촌점,한식,서울특별시 송파구 백제고분로40길 5,37.504004,127.106207
12107,남도에서,한식,서울특별시 강동구 동남로81길 58,37.559617,127.153701


In [181]:
# 업종명 종류 확인
df['food'].unique()

array(['경양식', '한식', '기타', '일식', '중국식', '분식', '뷔페식', '호프/통닭', '감성주점',
       '외국음식전문점(인도,태국등)', '출장조리', '복어취급', '패밀리레스트랑', '정종/대포집/소주방',
       '식육(숯불구이)', '횟집', '냉면집', '탕류(보신용)', '라이브카페', '김밥(도시락)', '키즈카페'],
      dtype=object)

In [182]:
# 업종별 개수 확인
len(df.loc[df['food'] == '키즈카페'])

2

In [183]:
# 업종별 자료 확인
df.loc[df['food'] == '경양식']

,place_name,food,address,lat,lng
0,커먼맨션,경양식,서울특별시 종로구 경희궁길 36,37.572920,126.970063
18,테이스트메이드,경양식,서울특별시 종로구 필운대로1길 8,37.577471,126.968960
23,스푼앤포크 타파스,경양식,서울특별시 도봉구 삼양로154길 11,37.655630,127.013892
37,미니무드(mini mood),경양식,서울특별시 용산구 후암로28마길 17,37.549988,126.980381
39,다이닝룸 뒨뉘,경양식,서울특별시 종로구 율곡로 82,37.577176,126.988167
...,...,...,...,...,...
12049,피카 청담,경양식,서울특별시 강남구 학동로88길 12,37.518353,127.052660
12073,애니오케이션 카페,경양식,서울특별시 강남구 도산대로51길 14,37.524057,127.038075
12076,유어네이키드치즈 압구정점,경양식,서울특별시 강남구 압구정로 343,37.528475,127.040053
12092,스피카(SPICA),경양식,서울특별시 강남구 도산대로37길 20,37.522278,127.031488


In [184]:
# 업종명 변경
df.replace('호프/통닭', '치킨', inplace=True)
df.replace('중국식', '중식', inplace=True)
df.replace('김밥(도시락)', '김밥전문점', inplace=True)
df.replace('키즈카페', '카페/디저트', inplace=True)
df.replace('횟집', '회', inplace=True)
df.replace('식육(숯불구이)', '육류/고기', inplace=True)
df.replace('냉면집', '냉면전문점', inplace=True)
df.replace('복어취급', '한식', inplace=True)

# 가게명을 바탕으로 업종명 변경
df.loc[df['place_name'].str.contains('치킨'), 'food'] = '치킨'
df.loc[df['place_name'].str.contains('족발'), 'food'] = '족발/보쌈'
df.loc[df['place_name'].str.contains('사이공'), 'food'] = '베트남음식점'
df.loc[df['place_name'].str.contains('카페'), 'food'] = '카페/디저트'
df.loc[df['place_name'].str.contains('본죽'), 'food'] = '죽'
df.loc[df['place_name'].str.contains('아이스크림'), 'food'] = '카페/디저트'
df.loc[df['place_name'].str.contains('커피'), 'food'] = '카페/디저트'
df.loc[df['place_name'].str.contains('디저트'), 'food'] = '카페/디저트'
df.loc[df['place_name'].str.contains('김밥'), 'food'] = '김밥전문점'
df.loc[df['place_name'].str.contains('베트남'), 'food'] = '베트남음식점'
df.loc[df['place_name'].str.contains('퍼'), 'food'] = '베트남음식점'
df.loc[df['place_name'].str.contains('버거'), 'food'] = '패스트푸드'
df.loc[df['place_name'].str.contains('튀김'), 'food'] = '분식'
df.loc[df['place_name'].str.contains('떡볶이'), 'food'] = '분식'
df.loc[df['place_name'].str.contains('쌀국수'), 'food'] = '베트남음식점'
df.loc[df['place_name'].str.contains('피자'), 'food'] = '피자'
df.loc[df['place_name'].str.contains('통닭'), 'food'] = '치킨'
df.loc[df['place_name'].str.contains('투썸플'), 'food'] = '카페/디저트'
df.loc[df['place_name'].str.contains('스타벅스'), 'food'] = '카페/디저트'
df.loc[df['place_name'].str.contains('이디야'), 'food'] = '카페/디저트'
df.loc[df['place_name'].str.contains('감자탕'), 'food'] = '한식'
df.loc[df['place_name'].str.contains('스시'), 'food'] = '일식'
df.loc[df['place_name'].str.contains('마라탕'), 'food'] = '중식'
df.loc[df['place_name'].str.contains('정육'), 'food'] = '육류/고기'
df.loc[df['place_name'].str.contains('숯불'), 'food'] = '육류/고기'
df.loc[df['place_name'].str.contains('곱창'), 'food'] = '육류/고기'
df.loc[df['place_name'].str.contains('지코바'), 'food'] = '치킨'
df.loc[df['place_name'].str.contains('횟집'), 'food'] = '회'

In [185]:
# 변경된 업종명 확인
df['food'].unique()

array(['경양식', '한식', '기타', '일식', '중식', '분식', '뷔페식', '치킨', '감성주점',
       '외국음식전문점(인도,태국등)', '출장조리', '패밀리레스트랑', '정종/대포집/소주방', '육류/고기', '회',
       '냉면전문점', '탕류(보신용)', '라이브카페', '김밥전문점', '카페/디저트'], dtype=object)

In [201]:
# 배달이 아닌 업종 제외하여 result 변수에 저장
result = df.loc[(df['food'] != '정종/대포집/소주방') & 
                (df['food'] != '뷔페식') & 
                (df['food'] != '감성주점') &
                (df['food'] != '출장조리') & 
                (df['food'] != '패밀리레스트랑') & 
                (df['food'] != '탕류(보신용)') & 
                (df['food'] != '라이브카페')]

In [205]:
# 변경된 업종 최종 확인
result['food'].unique()

array(['경양식', '한식', '기타', '죽', '일식', '중식', '카페/디저트', '피자', '분식', '베트남음식점',
       '패스트푸드', '치킨', '김밥전문점', '육류/고기', '족발/보쌈', '외국음식전문점(인도,태국등)', '회',
       '냉면전문점'], dtype=object)

In [207]:
result

,place_name,food,address,lat,lng
0,커먼맨션,경양식,서울특별시 종로구 경희궁길 36,37.572920,126.970063
1,귀한돌짜장 매운갈비찜,한식,서울특별시 강북구 4.19로 55,37.646420,127.008572
2,할배골,한식,서울특별시 도봉구 도봉로 493-4,37.649833,127.034923
3,망우로스팅랩,기타,서울특별시 중랑구 용마산로118길 67,37.601238,127.104370
4,본죽&비빔밥 서울양원역점,죽,서울특별시 중랑구 양원역로 82,37.607469,127.105874
...,...,...,...,...,...
12104,청년어부 삼전점,일식,서울특별시 송파구 백제고분로 198,37.503938,127.088562
12105,진전복 삼계탕 잠실직영점,한식,서울특별시 송파구 올림픽로35가길 10,37.516940,127.102633
12106,알이랑 고기랑 석촌점,한식,서울특별시 송파구 백제고분로40길 5,37.504004,127.106207
12107,남도에서,한식,서울특별시 강동구 동남로81길 58,37.559617,127.153701


### 5. CSV 저장

In [204]:
result.to_csv('place_location_final.csv', index=False, encoding='utf-8-sig')